In [1]:
import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"

In [25]:
import netket as nk

N = 16
hi = nk.hilbert.Spin(s=1 / 2, N=N)

In [26]:
import jax
hi.random_state(jax.random.PRNGKey(0), 3)

Array([[-1.,  1., -1., -1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
         1., -1.,  1.],
       [-1.,  1.,  1., -1.,  1., -1., -1.,  1.,  1., -1., -1., -1., -1.,
        -1., -1., -1.],
       [-1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,
        -1., -1., -1.]], dtype=float32)

In [27]:
from netket.operator.spin import sigmax,sigmaz

H = -1*sigmaz(hi,0)*sigmax(hi, 1)
H += -1*sigmax(hi,N-3)*sigmaz(hi, N-2)*sigmaz(hi, N-1)
H += -1*sigmax(hi,N-2)*sigmax(hi, N-1)
H += sum([-1*sigmax(hi,i)*sigmaz(hi,(i+1))*sigmax(hi,i+2) for i in range(N-3)])

In [28]:
HX = -1 * sigmax(hi, 0) * sigmaz(hi, 1)
HX += -1 * sigmaz(hi, N - 3) * sigmax(hi, N - 2) * sigmax(hi, N - 1)
HX += -1 * sigmaz(hi, N - 2) * sigmaz(hi, N - 1)
HX += sum([-1 * sigmaz(hi, i) * sigmax(hi, (i + 1)) * sigmaz(hi, i + 2) for i in range(N - 3)])


In [29]:
sp_h=H.to_sparse()
sp_h.shape

(65536, 65536)

In [30]:
from scipy.sparse.linalg import eigsh

eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("eigenvalues with scipy sparse:", eig_vals)

E_gs = eig_vals[0]

eigenvalues with scipy sparse: [-16. -14.]


In [31]:
graph=nk.graph.Chain(length=N, pbc=False)
g = nk.graph.Hypercube(length=N, n_dim=1, pbc=False)
print(graph.translation_group())

PermutationGroup(elems=[Id()], degree=16)


In [33]:
import time
ma = nk.models.RBMModPhase(alpha=16)
sa = nk.sampler.MetropolisLocal(hilbert=hi)

# Optimizer
op = nk.optimizer.Adam(learning_rate=0.001)
# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.1)

# The variational state
vs = nk.vqs.MCState(sa, ma, n_samples=256)

# The ground-state optimization loop
gs = nk.VMC(
    hamiltonian=HX,
    optimizer=op,
    preconditioner=None,
    variational_state=vs)

start = time.time()
gs.run(out='RBMModPhase', n_iter=25000)
end = time.time()

print('### RBM calculation')
print('Has',vs.n_parameters,'parameters')
print('The RBM calculation took',end-start,'seconds')

 12%|█▏        | 2895/25000 [05:10<39:32,  9.32it/s, Energy=-11.9978-0.0044j ± 0.0052 [σ²=0.0069, R̂=1.0348]]


KeyboardInterrupt: 

In [34]:
import netket.nn as nknn
import flax.linen as nn
import jax.numpy as jnp
import numpy as np

class FFNN(nn.Module):
    @nn.compact
    def __call__(self, x):
        x = nn.Dense(features=5*x.shape[-1],
                     use_bias=True,
                     param_dtype=np.complex128,
                     kernel_init=nn.initializers.normal(stddev=0.01),
                     bias_init=nn.initializers.normal(stddev=0.01)
                    )(x)
        x = nknn.log_cosh(x)
        x = jnp.sum(x, axis=-1)
        return x

model = FFNN()
# We shall use an exchange Sampler which preserves the global magnetization (as this is a conserved quantity in the model)
sa = nk.sampler.MetropolisHamiltonian(hilbert=hi, hamiltonian=H)

# Construct the variational state
vs = nk.vqs.MCState(sa, model, n_samples=1024)

# We choose a basic, albeit important, Optimizer: the Stochastic Gradient Descent
opt = nk.optimizer.Adam(learning_rate=0.0005)

# Stochastic Reconfiguration
sr = nk.optimizer.SR(diag_shift=0.01)

# We can then specify a Variational Monte Carlo object, using the Hamiltonian, sampler and optimizers chosen.
# Note that we also specify the method to learn the parameters of the wave-function: here we choose the efficient
# Stochastic reconfiguration (Sr), here in an iterative setup
gs = nk.VMC(hamiltonian=HX, optimizer=opt, variational_state=vs, preconditioner=None)

In [35]:
gs.run(out="test", n_iter=1000)

  0%|          | 0/1000 [00:00<?, ?it/s]C:\Users\Administrator\anaconda3\lib\site-packages\plum\parametric.py:198: UserWarning: `plum.Val` is deprecated and will be removed in a future version. Please use `typing.Literal` instead.
  return original_meta.__call__(cls, *args, **kw_args)
  0%|          | 0/1000 [00:08<?, ?it/s, Energy=0.33+0.00j ± 0.14 [σ²=20.70, R̂=1.0091]]C:\Users\Administrator\anaconda3\lib\site-packages\plum\parametric.py:198: UserWarning: `plum.Val` is deprecated and will be removed in a future version. Please use `typing.Literal` instead.
  return original_meta.__call__(cls, *args, **kw_args)
100%|██████████| 1000/1000 [31:12<00:00,  1.87s/it, Energy=-10.31-0.01j ± 0.17 [σ²=31.30, R̂=1.0044]]      


(JsonLog('test', mode=write, autoflush_cost=0.005)
   Runtime cost:
   	Log:    1.9114036560058594
   	Params: 0.03217053413391113,)